# 0. Sales Forecasting for Retail Chain

This project builds a time series forecasting model to predict future sales by store and product category for a retail chain.

The goal is to generate accurate demand forecasts to optimize inventory planning and operations for the retail chain.

The project uses historical daily sales data from the Ecuadorian retail chain. You can get the data and more details [here](https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data)

I worked on 3 CSV files of data:
- **Base de Datos.csv** : Contains time series of the stores and the product families sales. It combines the data of the "train" and "test" files in the kaggle repo.
- **holidays_events.csv** : Is a meta data. This data is quite valuable to understand past sales, trend and seasonality. It relevance is evaluated here
- **stores.csv** : meta data, gives some information about stores such as city, state, type, cluster.

# 1. Import of Data and Libraries used

In [ ]:
## Are you using Colab and you want the data to be stored in Drive? Uncomment this!

#from google.colab import drive
#drive.mount('/content/drive')
#%pip install -U ydata-profiling==4.1.1

## usage note: after running this cell, ydata-profiling may still not work.
## What needs to be done is restart the runtime environment and run this queue again from the first cell

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ydata_profiling as pp
from ydata_profiling import ProfileReport
import statsmodels.api as sm

In [ ]:
# Data Bases
path="/content/drive/MyDrive/Colab Notebooks/"
dfSales = pd.read_csv(path+"Base de Datos.csv")
dfStores = pd.read_csv(path+"stores.csv")
dfHolidays = pd.read_csv(path+"holidays_events.csv")

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
## Metrics
from sklearn import metrics

In [ ]:
import statsmodels.tsa.api as smtsa
import statsmodels.stats as smst
import statsmodels.stats.api as sms
from statsmodels.tsa.statespace.sarimax import SARIMAX

# 2. Exploratory Data Analysis

## 2.1 Data Preview

In [ ]:
dfSales.head(10)

In [ ]:
dfStores.head(10)

In [ ]:
dfHolidays.head(10)

## 2.2 EDA (Profiling)

Many standard exploratory data analysis tasks are automatically implemented in the `ydata_profiling` library.

This in turn provides information that allows certain particular analyzes of the work in question to be made.

**Things observed in this database**
1. Uniqueness of variable values of type *id*
2. Presence of missing data
3. Correct specification of categories in categorical type variables (e.g.:
"Administrator" and "admin" categories. must be unified in the same criterion)
4. Ranges of values taken by numerical variables
5. Consistency of information provided by variables in different data table bases

In [ ]:
ProfileReport(dfSales)

In [ ]:
ProfileReport(dfStores)

In [ ]:
ProfileReport(dfHolidays)

## 2.3 Parsing variables **date** (*str*) to datatype **DateTime**

In [ ]:
# Ejemplo
dfSales.loc[0,"date"] , dfHolidays.loc[0,"date"] # is type string, it is convenient that it be type DateTime

In [ ]:
# Variable Type Conversion
dfSales["dt_sales_date"]=pd.to_datetime(dfSales["date"])
dfHolidays["dt_holiday_date"] = pd.to_datetime(dfHolidays["date"])

#Column Rearrangement
dfSales = dfSales.reindex(columns=['id','date','dt_sales_date','store_nbr','family','sales','onpromotion'])
dfHolidays = dfHolidays.reindex(columns=['date','dt_holiday_date','type','locale','locale_name','description','transferred'])


In [ ]:
# Ejemplo
dfSales.loc[0,"dt_sales_date"] , dfHolidays.loc[0,"dt_holiday_date"]

## 2.4 Value range of DateTime type variables

The "holiday" attribute assigned to a date can, a priori, have a significant impact on the number of potential customers by increasing the number of people in circulation and thus on the volume of sales made on that date.

For this reason, the sales record period **must be contained** in the period in which the holidays are recorded.

In [ ]:
print("Periodo del registro de ventas")
print("inicio: " , dfSales["dt_sales_date"].min())
print("fin: " , dfSales["dt_sales_date"].max())


In [ ]:
print("Periodo del registro de holidays")
print("inicio: " , dfHolidays["dt_holiday_date"].min())
print("fin: " , dfHolidays["dt_holiday_date"].max())


## 2.5 Review Correct Specification of Categories

It is checked that the different categories of the various categorical variables represent different things, without being typing errors or equivalent expressions.

In [ ]:
# Categorías de Productos Tabla Sales
print("Sales - Familias de Productos (" , len(dfSales["family"].unique()) , ")")
print(dfSales["family"].unique())

In [ ]:
# Categorías Ciudades y Estados Tabla Stores
print("Stores - Ciudades (" , len(dfStores['city'].unique()) , ")")
print(dfStores['city'].unique())
print(" ")
print("Stores - Estados (" , len(dfStores['state'].unique()) , ")")
print(dfStores['state'].unique())

In [ ]:
# Categorias Locale Name Tabla Holidays
print("Holidays - Locale Names (" , len(dfHolidays['locale_name'].unique()) , ")" )
print(dfHolidays['locale_name'].unique())

# 3. Creation of a unified and Clean Database

Depending on the type of relationship between the records of the different Dataframes, I used different criteria to unify the information in the tables.

1. In the case of Sales & Stores tables:
The relationship between the records is many-to-one (1:n) with which a **merge** of the tables was made, without loss of information from any of them

2. The case of the Sales & Holidays tables:
The relationship is many-to-many. with which a merge is *NOT* implemented, since that creates new rows (sales) artificially and erroneously.
This happens because there are several holidays on the same day.
To unify the records, criteria must be established that preserve the information that is relevant to the Forecasting problem.
The information that interests me is **if the operation (sale) carried out that day, in that store, located in that place, was on a holiday of some type (local, regional, national)** In turn, I assign a order of importance to the holidays `national > regional > local` and I assume that all holidays of the same level have the same value (this supporting my assumption that the <font color='red'>importance of the holidays lies in the impact on the mass of circulation of potential clients on the day</font>) So, **if 2 holidays coincide on a date, I stick with the one with the highest level of importance and that occurs in the region/locality in which it was made the sale**

In [ ]:
# UNION TABLAS SALES & HOLIDAY (variable : store_nbr) (relación many-to-one)
dfSalesStores = dfSales.merge(right=dfStores , how='inner', left_on="store_nbr", right_on="store_nbr",suffixes=(None, "_store"),validate="many_to_one")
# Renombrar columnas para facil identificacion
dfSalesStores = dfSalesStores.rename(columns={'family':'sale_family', 'id':'sale_id', 'onpromotion':'sales_onpromotion', 'type':'store_type', 'cluster':'store_cluster','city': 'store_city','state': 'store_state'})
dfSalesStores.drop('sale_id', axis=1, inplace=True)
dfSalesStores

In [ ]:
# UNION TABLAS SALES & HOLIDAY
#creo primero las columnas que voy a agregar al DFSalesStores
dfSalesStores['was_Holiday_national_here'] = 0
dfSalesStores['was_Holiday_regional_here'] = 0
dfSalesStores['was_Holiday_local_here'] = 0

dfSalesStores['Holiday_locale'] = np.nan
dfSalesStores['Holiday_locale_name'] = np.nan

In [ ]:
# Algorithm that implements the union criterion explained above
k=0
fechas_en_holiday_df = set(dfHolidays['date'].values)
for index, row in dfSalesStores.iterrows():
  k=k+1
  print(f"{k}")
  date = row['date']
  state = row['store_state']
  city = row['store_city']

  was_hol_nac_here=0
  was_hol_reg_here=0
  was_hol_loc_here=0
  holiday_locale = np.nan
  holiday_locale_name = np.nan
  holiday_decription = np.nan

  if date in fechas_en_holiday_df:

    sub_dfHolidays= dfHolidays[dfHolidays['date'] == date]
    sub_alcanceDeFeriados = set(sub_dfHolidays['locale'].values)
    #print(sub_alcanceDeFeriados)
    if 'National' in sub_alcanceDeFeriados:
      sub_sub_dfHolidays=sub_dfHolidays[(sub_dfHolidays['locale']=='National') & (sub_dfHolidays['transferred']==False)]
      sub_sub_dfHolidays=sub_sub_dfHolidays.reset_index(drop=True)
      if sub_sub_dfHolidays.empty:
        a=1
      else:
          was_hol_nac_here=1
          holiday_locale = 'National'
          holiday_locale_name = sub_sub_dfHolidays.loc[0, 'locale_name']
          #holiday_decription = sub_sub_dfHolidays[0,'description']

    if ('National' not in sub_alcanceDeFeriados) & ('Regional' in sub_alcanceDeFeriados):
        sub_sub_dfHolidays=sub_dfHolidays[(sub_dfHolidays['locale']=='Regional') & (sub_dfHolidays['transferred']==False) & (sub_dfHolidays['locale_name']==state)]
        sub_sub_dfHolidays=sub_sub_dfHolidays.reset_index(drop=True)
        if sub_sub_dfHolidays.empty:
          a=1
        else:
          was_hol_reg_here=1
          holiday_locale = 'Regional'
          holiday_locale_name = sub_sub_dfHolidays.loc[0, 'locale_name']
          #holiday_decription = sub_sub_dfHolidays[0,'description']



    if ('National' not in sub_alcanceDeFeriados) & ('Regional' not in sub_alcanceDeFeriados) & ('Local' in sub_alcanceDeFeriados):
        sub_sub_dfHolidays=sub_dfHolidays[(sub_dfHolidays['locale']=='Local') & (sub_dfHolidays['transferred']==False) & (sub_dfHolidays['locale_name']==city)]
        sub_sub_dfHolidays=sub_sub_dfHolidays.reset_index(drop=True)
        if sub_sub_dfHolidays.empty:
          a=1
        else:
          was_hol_loc_here=1
          holiday_locale = 'Local'
          holiday_locale_name = sub_sub_dfHolidays.loc[0, 'locale_name']
          #holiday_decription = sub_sub_dfHolidays[0,'description']


    dfSalesStores.loc[index,'was_Holiday_national_here'] = was_hol_nac_here
    dfSalesStores.loc[index,'was_Holiday_regional_here'] = was_hol_reg_here
    dfSalesStores.loc[index,'was_Holiday_local_here'] = was_hol_loc_here

    dfSalesStores.loc[index,'Holiday_locale'] = holiday_locale
    dfSalesStores.loc[index,'Holiday_locale_name'] = holiday_locale_name
    #dfSalesStores2.loc[index,'Holiday_description'] = holiday_decription




In [ ]:
# I create variable "Was it a holiday?" (regardless of which type)

dfSalesStores['was_Holiday_here'] = np.zeros(dfSalesStores.shape[0])
for index, row in dfSalesStores.iterrows():
  if (row['was_Holiday_local_here']==1) | (row['was_Holiday_national_here']==1) | (row['was_Holiday_regional_here']==1):
    dfSalesStores.loc[index,'was_Holiday_here'] = 1

# Save unified DF
# BE CAREFUL OVERWRITE WHEN UNCOMMENTING
#dfSalesStores.to_csv("/content/drive/MyDrive/Colab Notebooks/baseDatosUnif_guardado.csv",index=False)

In [ ]:
# Reading from Unified Database created in previous cell
df_unificado = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/baseDatosUnif_guardado.csv")
df_unificado

# 4. Charts with relevant Sales information

## 4.1 How is the percentage of promotional sales over the total distributed by category?

The usefulness is to give an indicative measure of "in which categories the offers have a sales driver."

And it also allows you to have an idea of whether the use of promotions is a determining factor when estimating future sales.

In these cases, sales from promotions represent a very small fraction of the total, therefore, when modeling for forecasting, it is very likely that they will not be evident as a relevant (significant) variable.

In [ ]:
df1=df_unificado.groupby(['sale_family'])['sales','sales_onpromotion'].sum().reset_index()
df1["sales_no_prom_perc"] = 100*(df1["sales"]-df1["sales_onpromotion"]) / (df1['sales'])
df1["sales_on_prom_perc"] = 100*(df1["sales_onpromotion"]) / (df1['sales'])

In [ ]:
lista_families_sales = list(df_unificado['sale_family'].unique())

fig, ax = plt.subplots(figsize=(10,6))

ax.bar(df1['sale_family'], df1['sales_no_prom_perc'], label='Ventas Regulares')
ax.bar(df1['sale_family'], df1['sales_on_prom_perc'], bottom=df1['sales_no_prom_perc'], label='Ventas Promo')

ax.set_ylabel('Porcentaje')
ax.set_title('Ventas por Categoría')
ax.legend()
# Rotar los ticks del eje x
plt.xticks(rotation=90)
plt.yticks(range(0,105,5))


plt.show()

## 4.2 Sales Distribution based on days of the week

It serves to improve the management of resources and personnel according to the days of greatest demand.

Furthermore, if we find a clear pattern, it gives clues to find periodic trends in the sales series.

Surprisingly, Mondays are the 3rd day with the most sales.

In [ ]:
df_unificado['dt_sales_date'] = pd.to_datetime( df_unificado['date'])
df_unificado['day_name'] = df_unificado['dt_sales_date'].dt.day_name()

In [ ]:
df2=df_unificado.groupby(['day_name'])['sales'].mean().reset_index()
total = df2.sales.sum()
fig, ax = plt.subplots(figsize=(10,10))

ax.bar(df2['day_name'], 100*df2['sales']/total)

In [ ]:
df2

## 4.3 Monthly Percentage Growth of average sales by product category

These graphs will be particularly useful in sales forecasting for each category by facilitating the identification of cyclical and seasonal components.

Clear example: school supplies

In [ ]:
df_unificado['date'] = pd.to_datetime(df_unificado['date'])
df_unificado['year'] = df_unificado['date'].dt.year
df_unificado['month'] = df_unificado['date'].dt.month

# Lista de familias a visualizar
familias = df_unificado['sale_family'].unique()

# Crea la figura con subgráficos
fig, axes = plt.subplots(nrows=33, ncols=1, figsize=(10, 200))

# Itera sobre las familias y crea los subgráficos
for i, familia in enumerate(familias):
    # Filtra el DataFrame para la familia actual
    df_familia = df_unificado[df_unificado['sale_family'] == familia]
    df_familia = df_familia.groupby(['year','month'])['sales'].mean().reset_index()
    df_familia['growth'] = df_familia['sales'].pct_change() * 100
    df_familia['growth']=df_familia['growth'].fillna(0)

    ax = axes[i]

    # Crea el gráfico en el axis actual
    ax.plot( df_familia['year'].astype(str) + '-' + df_familia['month'].astype(str), df_familia['growth'], marker='o', linestyle='-', color='b')
    ax.set_title(f'Crecimiento Porcentual Mensual - {familia}')
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Crecimiento Porcentual Mensual')
    ax.grid(True)
    ax.tick_params(axis='x', rotation=90)
    ax.axhline(y=0.0, color='r')

# Ajusta el diseño de la figura y muestra la misma
plt.tight_layout()
plt.show()

## 4.4 Estimation of the Impact of Holidays on sales

We seek to see if the "Holiday" condition has a significant effect on the sales that usually take place that day of the week (MON, TUE, WED, THU, FRI, SAT, SUN) this is complemented with what was said in **Sec 4.2**

I found that holidays do have a significant impact, with those on Mondays and Wednesdays being very similar to those on weekends!

In [ ]:
df_unificado['was_Holiday_here'] = df_unificado['was_Holiday_here'].astype(int)
df_unificado['dt_sales_date'] = pd.to_datetime( df_unificado['date'])
df_unificado['day_name'] = df_unificado['dt_sales_date'].dt.day_name()


In [ ]:
df_Hol = df_unificado[df_unificado['was_Holiday_here'] == 1]
df_NoHol = df_unificado[df_unificado['was_Holiday_here'] != 1]
print(df_Hol.shape)
print(df_NoHol.shape)

In [ ]:
df_Hol_day=df_Hol.groupby(['day_name'])['sales'].mean().reset_index()
df_NoHol_day=df_NoHol.groupby(['day_name'])['sales'].mean().reset_index()

In [ ]:
df=pd.DataFrame()
df['day_name'] = df_Hol_day['day_name']
df['sales_Hol_day'] = df_Hol_day['sales']
df['sales_NoHol_day'] = df_NoHol_day['sales']
df

In [ ]:
ax = df.plot.bar(x='day_name', y=['sales_Hol_day', 'sales_NoHol_day'], rot=0)

# Agregar etiquetas de eje y título
ax.set_xlabel('Días de la semana')
ax.set_ylabel('Ventas')
ax.set_title('Ventas medias por día de la semana')
ax.set_yticks(np.arange(0,550,50))
ax.axhline(y=df['sales_Hol_day'].mean(),color='#1f77b4',linestyle='dotted',label="Mean sales_Hol_day")
ax.axhline(y=df['sales_NoHol_day'].mean() ,color='#ff7f0e',linestyle='dotted',label="Mean sales_NoHol_day")
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
# Mostrar el gráfico
plt.show()

# 5. Branch that sold the most

## 5.1 Which branch sold the most? (monthly sales)

I found that store 44 is the one that sells the most monthly for all months

- This store is located in **Pichincha, Quito**
- It is type **A** and belongs to Cluster **5**

In [ ]:
dfStores[dfStores['store_nbr']==44]

In [ ]:
# Función Plot las 10 tiendas con mayores ventas por mes en un dado año
def most_sales_stores(df,ano):
  df_ano = df[df['year']==ano]
  top10_stores_by_month = df_ano.groupby(['month','store_nbr'])['sales'].sum().reset_index()
  top10_stores_by_month = top10_stores_by_month.sort_values(['month','sales'], ascending=[True,False]).groupby('month').head(10)

  # Crear la figura con 12 subplots
  fig, axs = plt.subplots(3, 4, figsize=(12,8), sharey=True)

  # Graficar cada mes
  for i, month in enumerate(top10_stores_by_month['month'].unique()):
    ax = axs[i//4, i%4]
    sales = top10_stores_by_month[top10_stores_by_month['month']==month]
    ax.bar(sales['store_nbr'].map(str), sales['sales'])
    ax.set_title(f'Mes {month}')
  plt.tight_layout()
  plt.show()

In [ ]:
#Creación de Variables Auxiliares Extra
dfSales['year'] = dfSales['dt_sales_date'].dt.year
dfSales['month'] = dfSales['dt_sales_date'].dt.month
dfSales['week'] = dfSales['dt_sales_date'].dt.week

In [ ]:
# Tiendas con mas ventas 2013
most_sales_stores(dfSales,2013)

In [ ]:
# Tiendas con mas ventas 2014
most_sales_stores(dfSales,2014)

In [ ]:
# Tiendas con mas ventas 2015
most_sales_stores(dfSales,2015)

In [ ]:
# Tiendas con mas ventas 2016
most_sales_stores(dfSales,2016)

In [ ]:
# Tiendas con mas ventas 2017
most_sales_stores(dfSales,2017)

## 5.2 Does Store 44 sell all products?

It doesn't sell BABY CARE, so by direct inference,

**No forecasting is done for this category**

In [ ]:
# DATAFRAME VENTAS TIENDA 44
dfSales44=df_unificado[(df_unificado['store_nbr']==44)].reset_index(drop=True)

In [ ]:
# Totalidad de productos vendidos
dfSales44.groupby(['sale_family'])['sales'].sum().reset_index()

In [ ]:
dfSales44=dfSales44[(dfSales44['sale_family'] != "BABY CARE")].reset_index(drop=True)


#6. Total Store Sales Forecasting – Autoregressive Model

When forecasting, the question arises: How to treat time series? Do we group the data by day/week/month and make predictions by day/week/month?

**I apply SARIMA autoregressive models to make the predictions**W

It must be taken into account that, in general, the higher the level of granularity, the greater the irregularities and the worse the forecasts are (especially at 12 months).

At the other extreme, making monthly predictions (high level of aggregation) may not be useful in more frequently consumed items such as categories: **Grocery** & **Prepared Foods** among others.

That is why, as a general criterion, I consider that the best option would be a weekly sales forecast, complemented with a daily distribution of that weekly estimate following the results of **Sec 4.2. and Sec 4.4** (depending on the existence of holidays) for each category. However, I will see the approach I choose category by category

<font color='red'> Metric used in the evaluation of the models </font> I use the **RMSE** metric because it is a direct measure of the errors in the estimate calculated from the variance. Furthermore, being sensitive to outliers, it penalizes models in which some estimates differ greatly (which is precisely a problem that we would not want in sales forecasting).

This metric is not good when working with quantities that vary by many orders of magnitude (small values are associated with a very large mean error due to the presence of very large values in the calculation of the metric).

In this case of forecasting, the latter does not represent a major problem with this data.

<font color='red'> Something that is very useful is to consider the RMSE/(series mean) this provides a dimensionless comparison point to compare the different regressions. </font>

** General considerations: I will assume that the forecast of a model is optimal and acceptable if RMSE represents approximately 1/4 or 1/5 of the mean of the series (that amount seems acceptable to me as a measure of error to continue obtaining significant predictions with its associated error). And, also check if the requirement that the prediction residuals behave like White Noise is met**


**Note** The most important factor when studying time series is to analyze their autocorrelation

In [ ]:
## Definición funciones útiles
# Función Gráfico ACF y PACF
def my_ts_plot(serie, titulo, displayLags=30):
    fig = plt.figure()
    fig.set_figheight(8)
    fig.set_figwidth(8)

    fig.suptitle(titulo, fontsize = 14, fontweight = "bold")
    plt.subplots_adjust(top = 0.95)
    ax0 = plt.subplot2grid(shape=(2, 2), loc=(0, 0), colspan=2)
    ax1 = plt.subplot2grid(shape=(2, 2), loc=(1, 0), colspan=1)
    ax2 = plt.subplot2grid(shape=(2, 2), loc=(1, 1), rowspan=1)

    ax0.plot(serie)
    sm.graphics.tsa.plot_acf(serie, lags=displayLags, ax=ax1)
    sm.graphics.tsa.plot_pacf(serie, lags=displayLags, ax=ax2)

    plt.show()

def scores_models(y_real , y_predicted):
    MSE = metrics.mean_squared_error(y_real, y_predicted)
    RMSE = np.sqrt( metrics.mean_squared_error(y_real, y_predicted) )
    MAPE = metrics.mean_absolute_percentage_error(y_real,y_predicted)
    R2score = metrics.r2_score(y_true=y_real,y_pred=y_predicted)

    print(f'Score MSE  : {MSE:0.5f}')
    print(f'Score RMSE : {RMSE:0.5f}')
    #print(f'Score MAPE : {MAPE:0.5f}')
    print(f'R^2-score  : {R2score:0.5}')

In [ ]:
dfSales44['dt_sales_date'] = pd.to_datetime( dfSales44['date'])
dfSales44['year'] = dfSales44['dt_sales_date'].dt.year
dfSales44['month'] = dfSales44['dt_sales_date'].dt.month
dfSales44['week'] = dfSales44['dt_sales_date'].dt.week

dfSales44['count'] =  np.arange(len(dfSales44.index))

## 6.1 Weekly (Total) Sales Scheme

In [ ]:
# Agrupar por 'year' y 'week' , sumando las ventas
dfSales44_semanal = dfSales44.groupby(['year', 'week'])['sales'].sum().reset_index()
dfSales44_semanal = dfSales44_semanal.drop(dfSales44_semanal.index[-1])
dfSales44_semanal = dfSales44_semanal.drop(dfSales44_semanal.index[-1])
dfSales44_semanal['count'] =  np.arange(len(dfSales44_semanal.index))
dfSales44_semanal

In [ ]:
# Crear un gráfico de serie temporal
plt.figure(figsize=(60, 6))


plt.plot(dfSales44_semanal['year'].astype(str) + '-' + dfSales44_semanal['week'].astype(str), dfSales44_semanal['sales'])

plt.title('Serie Ventas Semanales Tienda 44')
plt.xlabel('Año-Mes')
plt.ylabel('Ventas Totales')

# Colocar la leyenda fuera del área del gráfico
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

# Rotar los ticks del eje x
plt.xticks(rotation=90)

plt.show()

### 6.1.1 Series Components

We can see that the series has
- A trend component, which I will model as a **linear trend**

- A periodic component with a periodicity of 12 months (52 weeks) as suggested by the series in the graph in the previous cell (peak at the end of the year and a drop in consumption in the first week of the following year)

In [ ]:
resultados = sm.tsa.seasonal_decompose(dfSales44_semanal['sales'], model='additive', period=52)

resultados.plot()

plt.show()

In [ ]:
# Grafico Tendencia Lineal
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(30,8))

# Serie Sales Semanal
axes.plot('count', 'sales', data=dfSales44_semanal)
axes.set_title("Tendencia Lineal", fontsize=20)
# Recta de Ajuste Tendencia
axes = sns.regplot(x='count',y='sales',data=dfSales44_semanal)

plt.show()

In [ ]:
my_ts_plot(dfSales44_semanal['sales'], 'Serie Ventas Semanal',60) # No es estacionario obviamente como ya vimos

In [ ]:
#diferenciamos el proceso <-- Para garantizar estacionaridad
diff_dfSales44_semanal = np.diff(dfSales44_semanal['sales'])

In [ ]:
my_ts_plot(diff_dfSales44_semanal, 'Serie Ventas Semanal Diferenciada',60)
#Se encuentra una componente de estacionalidad mostrando semejanza de la serie con su año anterior

### 6.1.2 Model

In [ ]:
# Modelo SARIMA
model = sm.tsa.statespace.SARIMAX(dfSales44_semanal['sales'], trend='ct', order=(1,0,1),seasonal_order=(2,1,0,52))
result = model.fit(disp=False)
residuos = result.resid
my_ts_plot(residuos, 'Residuos',60) # Análisis de Residuos, verificamos que se comporten como White Noice

In [ ]:
# Model evaluation

data= pd.DataFrame()
data['SerieOG'] = dfSales44_semanal['sales'].values
#data['dates'] = dfSales44_semanal['year'].astype(str) + "-" +  dfSales44_semanal['week'].astype(str)
#data['dates'] = pd.to_datetime(data['dates'] + '-1', format='%Y-%U-%w')
data['Forecast'] = result.predict(start = 10, end = 300, dynamic= False)
#data.set_index(data['dates'],inplace=True)

scores_models(data['SerieOG'].values , data['Forecast'].fillna(0).values)

data[['SerieOG', 'Forecast']].plot(figsize=(30, 8))
#data[['Forecast']].plot(figsize=(60, 8))

# Rotar los ticks del eje x
#plt.xticks(rotation=90)
plt.show()

In [ ]:
# Mismo grafico que antes pero con las fechas como índices
data= pd.DataFrame()
data['SerieOG'] = dfSales44_semanal['sales'].values
data['dates'] = dfSales44_semanal['year'].astype(str) + "-" +  dfSales44_semanal['week'].astype(str)
data['dates'] = pd.to_datetime(data['dates'] + '-1', format='%Y-%U-%w')
data['Forecast'] = result.predict(start = 10, end = 300, dynamic= False)
data.set_index(data['dates'],inplace=True)
data[['SerieOG', 'Forecast']].plot(figsize=(100, 8),xticks = data.index)
#data[['Forecast']].plot(figsize=(60, 8))

# Rotar los ticks del eje x
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Forecasting - Chart
fig, ax = plt.subplots(figsize=(15, 5))
dfSales44_semanal['sales'].plot(ax=ax)
fcast = result.get_forecast(steps=50).summary_frame()
fcast['mean'].plot(ax=ax, style='k--')
ax.fill_between(fcast.index, fcast['mean_ci_lower'], fcast['mean_ci_upper'], color='k', alpha=0.1);

### 6.1.3 Observation Important Behavior

Note that the sales estimate made over the time interval of the database, estimates with a certain notable similarity to the data available, **except in January ≈ January 2015 (data number 100)**

#7. Forecasting Sales of Each Category

A procedure scheme similar to the previous one is followed, but I will define functions to simplify repetitive tasks for all categories.

## 7.0 Definition of functions for forecasting by categories

In [ ]:
def crear_df_dela_categoria(df,familia):
  df_cat = df[df['sale_family'] == familia].reset_index(drop=True)
  df_cat['dt_sales_date'] = pd.to_datetime( df_cat['date'])
  df_cat['year'] = df_cat['dt_sales_date'].dt.year
  df_cat['month'] = df_cat['dt_sales_date'].dt.month
  df_cat['week'] = df_cat['dt_sales_date'].dt.week
  print(df_cat.shape)
  return df_cat


In [ ]:
def crear_serie_categoria(df_cat,tipo):
  if tipo == 'diaria':
    df_cat_agrup = pd.DataFrame()
    df_cat_agrup['sales'] = df_cat['sales'].reset_index(drop=True)
    df_cat_agrup['count'] =  np.arange(len(df_cat_agrup.index))

  if tipo == 'semanal':
    # Agrupar por 'year' y 'week' , sumando las ventas
    df_cat_agrup = df_cat.groupby(['year', 'week'])['sales'].sum().reset_index()
    df_cat_agrup = df_cat_agrup.drop(df_cat_agrup.index[-1]) #tiro la ultima fila que no se corresponde a ningun dato
    df_cat_agrup = df_cat_agrup.drop(df_cat_agrup.index[-1]) #tiro la ultima fila que es semana incompleta
    df_cat_agrup = df_cat_agrup.reset_index()
    df_cat_agrup['count'] =  np.arange(len(df_cat_agrup.index))

  if tipo == 'mensual':
    # Agrupar por 'year' y 'week' , sumando las ventas
    df_cat_agrup = df_cat.groupby(['year', 'month'])['sales'].sum().reset_index()
    df_cat_agrup = df_cat_agrup.drop(df_cat_agrup.index[-1]) #tiro la ultima fila que es mes incompleto
    df_cat_agrup = df_cat_agrup.reset_index()
    df_cat_agrup['count'] =  np.arange(len(df_cat_agrup.index))

  return df_cat_agrup[['count','sales']]

In [ ]:
# Crear un gráfico de serie la temporal
def graficar_serie_temporal(serie,familia):
  plt.figure(figsize=(60, 6))
  plt.plot(serie['count'],serie['sales'])
  plt.title('Serie Ventas categoría '+familia+' Tienda 44')
  plt.ylabel('Cantidad de Ventas')
  plt.xticks(rotation=90)
  plt.show()

In [ ]:
def analizar_componentes_y_tendencia(serie,modelo,periodo):
  resultados = sm.tsa.seasonal_decompose(serie['sales'], model=modelo, period=periodo)
  resultados.plot()
  plt.show()
  #---------------------------------------------------------------------------------------------------
  # Grafico Tendencia
  fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(30,8))
  axes.plot('count', 'sales', data=serie)
  axes.set_title("Tendencia ", fontsize=20)
  # Recta de Ajuste Tendencia
  axes = sns.regplot(x='count',y='sales',data=serie)
  plt.show()
#---------------------------------------------------------------------------------------------------

In [ ]:
def correlograma_serie_seriediff(serie,orden):
  #correlograma serie
  my_ts_plot(serie['sales'], 'Serie Ventas',orden)

  #diferenciamos el proceso <-- Para garantizar estacionaridad
  serie_diff = np.diff(serie['sales'])
  #correlograma serie diferenciada
  my_ts_plot(serie_diff, 'Serie Ventas Diferenciada',orden)


In [ ]:
def modelo_y_evaluacion(serie,tendencia,ord_tupla,sea_ord_tupla,orden_correlograma,forecast_end,forecast_steps):
  # Modelo SARIMA
  model = sm.tsa.statespace.SARIMAX(serie['sales'], trend=tendencia, order=ord_tupla,seasonal_order=sea_ord_tupla)
  result = model.fit()
  residuos = result.resid
  my_ts_plot(residuos, 'Residuos',orden_correlograma) # Análisis de Residuos, verificamos que se comporten como White Noice

  #---------------------------------------------------------------------------------------------------
  # Evaluación del modelo
  data= pd.DataFrame()
  data['SerieOG'] = serie['sales'].values
  data['Forecast'] = result.predict(start = 10, end = forecast_end, dynamic= False)

  scores_models(data['SerieOG'].values , data['Forecast'].fillna(0).values)

  data[['SerieOG', 'Forecast']].plot(figsize=(30, 8))

  plt.show()

  #---------------------------------------------------------------------------------------------------
  # Visualización Gráfica de Forecasting
  fig, ax = plt.subplots(figsize=(60, 5))
  serie['sales'].plot(ax=ax)
  fcast = result.get_forecast(steps=forecast_steps).summary_frame()
  fcast['mean'].plot(ax=ax, style='k--')
  ax.fill_between(fcast.index, fcast['mean_ci_lower'], fcast['mean_ci_upper'], color='k', alpha=0.1);
  #---------------------------------------------------------------------------------------------------

## 7.1 Categoría AUTOMOTIVE

In [ ]:
categoria='AUTOMOTIVE'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365 # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40 # tiene que ser acorde al tipo de serie que se eligio arriba
#tendencia_serie =  [1,1,1] # a+bt+ct^2+dt^3
tendencia_serie = 'ct'
forecast_end=2013
forecast_steps=365

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
# nota: (no tratado) parece que puede existir un comportamiento ciclico con una periodiciodad de 5 años aprox (para cosas de automotor tendría sentido)
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)


In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=(1,0,1),sea_ord_tupla=(1,1,1,7),orden_correlograma=orden_correlograma,forecast_end=forecast_end,forecast_steps=forecast_steps)

## 7.2 BEAUTY Category

When I have few sales, as in this case, sometimes it is convenient to do it weekly because otherwise the predictions are always exponentially decreasing.

In [ ]:
categoria='BEAUTY'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (0,1,1)
sarima_sea_ord_tupla = (1,0,0,7)
forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
sarima_ord_tupla = (1,1,0)
sarima_sea_ord_tupla = (1,0,1,4)
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end,forecast_steps=forecast_steps)

## 7.3 Categoría BEVERAGES

In [ ]:
categoria='BEVERAGES'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (3,1,0,7)
forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)


In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)



In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.4 BOOKS Category

These series have very few sales
At first, this category may not have been selling. That's why I delete the first records

However, a clear trend is observed that sales of this type of product tend towards zero, which is reaffirmed with the forecasting.

**Note**: negative values in the forcasting of any category must be corrected ad hoc to zero sales.

In [ ]:
categoria='BOOKS'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 30               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (0,1,1)
sarima_sea_ord_tupla = (0,0,0,0)
forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
#Esta serie tiene muchos ceros y extremadamente pocas ventas. Elimino los primeros nulos

#Elimino los primeros registros nulos de la serie
serie_cat.drop(serie_cat.index[:1300], axis=0, inplace=True)
serie_cat=serie_cat.reset_index(drop=True)
serie_cat['count'] =  np.arange(len(serie_cat.index))
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end,forecast_steps=forecast_steps)

## 7.5 Categoría BREAD/BAKERY

In [ ]:
categoria='BREAD/BAKERY'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,0,0,7)
forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

Hay 3 aspectos a destacar:
1) Claramente se observa un comportamiento periódoco (más allá del semanal esperable) a lo largo de los años (se ve una caida a periodos regulares)

2) El correlograma de la ACF de la serie muestra que es bastante estable los patrones de compra semanales

3) La serie diaria tiene mucha variabilidad (ruido)

**es por ello que considero que una mejor alternativa en este caso sería optar por lo dicho en sección 6) y realizar un pronóstico de ventas semanal y distribuir las ventas semanales siguiendo un esquema como el de la figura de sección 4.2**

A continuación muestro los resultados de este metodo lo cuales parecen ser mejores que el del forecasting diario (viendo RMSE / mean )

In [ ]:
categoria='BREAD/BAKERY'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,52)
forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.6 CELEBRATION Category

This variable has very high increases in specific weeks of the year and few sales in general (they are not daily consumption items), so the best approach would be to estimate sales on a weekly basis.

In [ ]:
categoria='CELEBRATION'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (1,1,1,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
#Esta serie tiene muchos ceros, esto lo interpreto como que solo se ponian en stock los productos en determinadas epocas
#Esto afecta los pronósticos por la irregularidad de implica pr lo tanto y extremadamente pocas ventas.
#Elimino los primeros registros (aprox 900)

#Elimino los primeros registros nulos de la serie
#serie_cat.drop(serie_cat.index[:900], axis=0, inplace=True)
#serie_cat=serie_cat.reset_index(drop=True)
#serie_cat['count'] =  np.arange(len(serie_cat.index))
#graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.7 Categoría CLEANING

In [ ]:
categoria='CLEANING'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (2,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

Hay 3 aspectos a destacar:
1) Claramente se observa un comportamiento periódoco (más allá del semanal esperable) a lo largo de los años (se ve una caida a periodos regulares)

2) El correlograma de la ACF de la serie muestra que es bastante estable los patrones de compra semanales

3) La serie diaria tiene mucha variabilidad (ruido)

**es por ello que considero que una mejor alternativa en este caso sería optar por lo dicho en sección 6) y realizar un pronóstico de ventas semanal y distribuir las ventas semanales siguiendo un esquema como el de la figura de sección 4.2**

A continuación muestro los resultados de este metodo lo cuales parecen ser mejores que el del forecasting diario (viendo RMSE / mean )

In [ ]:
categoria='CLEANING'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,52)
forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.8 Categoría DAIRY

In [ ]:
categoria='DAIRY'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 7               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 20           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

Por lo mismo que dije en 7.7 uso un esquema semanal

In [ ]:
categoria='DAIRY'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 7               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.9 Categoría DELI

Tiene un comportamiento muy similar a CLEANING con lo cual la trato en forma similar

In [ ]:
categoria='DELI'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,1,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.10 Categoría EGGS

In [ ]:
categoria='EGGS'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = [1,0,1]
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.11 Categoría FROZEN FOODS

In [ ]:
categoria='FROZEN FOODS'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 55           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (2,0,1,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.12 Category GROCERY I

Something to note, it can be seen in the series that grocery sales took off between indices 150-200, which correspond to the earthquake in Ecuador in 2016 (I suppose)

Note: all the forecasts I try suggest a decrease in sales, contrary to the general trend of the series...

In [ ]:
categoria='GROCERY I'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 55           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (2,0,1)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)


In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.13 Categoría GROCERY II

In [ ]:
categoria='GROCERY II'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (3,0,0,7)


forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.14 HARDWARE Category
Given the few sales and high variability of this category, I analyze it weekly

In [ ]:
Dcategoria='HARDWARE'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.15 Categoría HOME AND KITCHEN I

In [ ]:
categoria='HOME AND KITCHEN I'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (3,1,1,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.16 Categoría HOME AND KITCHEN II

In [ ]:
categoria='HOME AND KITCHEN II'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (3,1,1,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.17 Categoría HOME APPLIANCES

In [ ]:
categoria='HOME APPLIANCES'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (2,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.18 Categoría HOME CARE

In [ ]:
categoria='HOME CARE'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (1,1,1,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
#Esta serie tiene muchos ceros y extremadamente pocas ventas. Elimino los primeros nulos

#Elimino los primeros registros nulos de la serie
serie_cat.drop(serie_cat.index[:880], axis=0, inplace=True)
serie_cat=serie_cat.reset_index(drop=True)
serie_cat['count'] =  np.arange(len(serie_cat.index))
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.19 Categoría LADIESWEAR

In [ ]:
categoria='LADIESWEAR'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.20 Categoría LAWN AND GARDEN

In [ ]:
categoria='LAWN AND GARDEN'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52              # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (3,1,0,7)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.21 LINGERIE Category

This variable also appears to have a cyclical component of some sort.

In [ ]:
categoria='LINGERIE'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (3,0,2)
sarima_sea_ord_tupla = (3,1,2,7)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.22 Categoría LIQUOR WINE BEER

In [ ]:
categoria='LIQUOR,WINE,BEER'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (3,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.23 Categoría MAGAZINES

In [ ]:
categoria='MAGAZINES'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,0)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.24 Categoría MEATS

In [ ]:
categoria='MEATS'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (2,1,1,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.25 Categoría PERSONAL CARE

In [ ]:
categoria='PERSONAL CARE'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,0)
sarima_sea_ord_tupla = (1,1,0,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.26 Categoría PET SUPPLIES

In [ ]:
categoria='PET SUPPLIES'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (3,1,2,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
#Esta serie tiene muchos ceros y extremadamente pocas ventas. Elimino los primeros nulos

#Elimino los primeros registros nulos de la serie
serie_cat.drop(serie_cat.index[:880], axis=0, inplace=True)
serie_cat=serie_cat.reset_index(drop=True)
serie_cat['count'] =  np.arange(len(serie_cat.index))
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.27 Categoría PLAYERS AND ELECTRONICS

In [ ]:
categoria='PLAYERS AND ELECTRONICS'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,1)
sarima_sea_ord_tupla = (3,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
#Esta serie tiene muchos ceros y extremadamente pocas ventas. Elimino los primeros nulos

#Elimino los primeros registros nulos de la serie
serie_cat.drop(serie_cat.index[:880], axis=0, inplace=True)
serie_cat=serie_cat.reset_index(drop=True)
serie_cat['count'] =  np.arange(len(serie_cat.index))
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.28 Categoría POULTRY

In [ ]:
categoria='POULTRY'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,0)
sarima_sea_ord_tupla = (3,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.29 Categoría PREPARED FOODS

In [ ]:
categoria='PREPARED FOODS'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,0,0)
sarima_sea_ord_tupla = (3,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.30 Categoría PRODUCE

In [ ]:
categoria='PRODUCE'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (2,1,0,7)

forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.31 Category SCHOOL AND OFFICE SUPPLIES

Sales of items of this variable skyrocket at a certain time, very high in specific weeks of the year and few sales in general, so the best approach would be to estimate sales on a weekly basis.

In [ ]:
categoria='SCHOOL AND OFFICE SUPPLIES'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (0,0,1)
sarima_sea_ord_tupla = (1,1,1,52)

forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)

## 7.32 Categoría SEAFOOD

In [ ]:
categoria='SEAFOOD'
tipo_serie = 'diaria'             #'diaria'  'semanal' 'mensual'
periodo_serie = 365               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 40           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (1,1,1)
sarima_sea_ord_tupla = (2,1,0,7)


forecast_end = 2013               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 365              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal


In [ ]:
categoria='SEAFOOD'
tipo_serie = 'semanal'             #'diaria'  'semanal' 'mensual'
periodo_serie = 52               # tiene que ser acorde al tipo de serie que se eligio arriba
modelo_componentes_serie = 'additive'
orden_correlograma = 60           # tiene que ser acorde al tipo de serie que se eligio arriba
tendencia_serie = 'ct'
sarima_ord_tupla = (0,1,1)
sarima_sea_ord_tupla = (1,1,1,7)


forecast_end = 300               # tiene que ser acorde al tipo de serie que se eligio arriba : 2013 diaria  , 300 semanal
forecast_steps = 52              # tiene que ser acorde al tipo de serie que se eligio arriba : 365 diaria  , 52 semanal

In [ ]:
df_categoria=crear_df_dela_categoria(dfSales44,categoria)
serie_cat = crear_serie_categoria(df_categoria,tipo_serie)
graficar_serie_temporal(serie_cat,categoria)

In [ ]:
analizar_componentes_y_tendencia(serie_cat,modelo_componentes_serie,periodo_serie)

In [ ]:
correlograma_serie_seriediff (serie_cat,orden_correlograma)

In [ ]:
modelo_y_evaluacion(serie=serie_cat,tendencia=tendencia_serie,ord_tupla=sarima_ord_tupla,sea_ord_tupla=sarima_sea_ord_tupla,orden_correlograma=orden_correlograma,forecast_end=forecast_end , forecast_steps=forecast_steps)